# BC205: Algorithms for Bioinformatics.
## II. Biological Sequence Analysis
### Christoforos Nikolaou



#### Biological Sequence Analysis
* Biological Sequence Analysis includes all lexicographic, statistical, or modeling types of analyses conducted on biological sequences such as DNA, RNA and proteins. In this class we will start looking into real biological problems, focusing on DNA sequences. 
* We will discuss some very basic concepts of computation such as hashing.
* We will then turn to the implementation of the things we learnt last time using Brute Force and Divide and Conquer Approaches.
* We will go back the concept of Binary Search and implement it the context of Genome Analysis.

#### The biological problems:  
* Compare different species on the basis of DNA composition
* Find evidence of horizontal gene transfer in a bacterial genome
* Locate the Origin of Repication of a Bacterial Genome

#### Bioinformatics Warm-Up
1. You are given a DNA sequence
    * Can you count the number of nucleotides of each of the four bases (A, G, C, T)?
    * How many calculations will you need?
    * How will you implement it?
2. Now consider the same problem only instead of nucleotides we need to count the number of all octanucleotides. What do you need to consider to attack the problem? What is different between the 4 mononucleotides and the 8^4 octanucleotides?

#### Aspects of DNA Composition
* GC content
* k-mer frequencies
* genomic signatures
* parity distributions


#### GC content
We call GC content (or GC%) the ratio of (G+C) nucleotides of a given DNA sequence
* Why is it important? G-C pairs are linked with 3 hydrogen bonds, while A-T ones with 2. High GC genomes are more stable in terms of physical chemistry.

![GC Content](figures/GCcontent.jpeg)


#### GC is related to:
* Biochemical level: Thermal stability
* Evolutionary level: Organism Phylogeny, Mutational pressures
* Genomic level: Genome size
* Functional level: Functional role of underlying sequences
* and many more

#### GC content in Genomic Sequences
* Bacteria: GC% is highly variable **between** species
* Bacteria: GC% is rather homogeneous **within** each genome
* Bacteria: GC% can be used in their classification

![GC Content](figures/GCcontentTree.jpeg)

#### GC content in Genomic Sequences
* Eukaryotes: Very homogeneous overall GC% (~40-45% in all animals)
* Eukaryotes: Fluctuation of GC contentalong the chromosomes and organization in areas of (rather) stable GC%
* Eukaryotes: Regions of stable high/low GC content that segregate mammalian genomes in isochores


#### Problem 1: GC content in Bacterial Genomes
* Given the DNA sequence of a Bacterial Genome, calculate its GC content:
    * Read the Sequence
    * Enumerate G
    * Enumerate C
    * Divide (G+C) over length of the sequence

#### GCContent. Pseudocode
* Τhe idea is to **exhaustively** enumerate all mononucleotides, therefore our approach is a very basic Brute Force approach.
* Given that the content of the sequence is unknown we have no other choice.
* We will proceed by reading each nucleotide in the sequence and check its value. Then increment a variable each time we find a G or a C.

#### GCContent: Implementation (naive)


In [ ]:
# Naive GC content

def naiveGC(genomefile):
    file = open(genomefile, 'r')

    seq = ""
    window = 1000
    total = 0
    nG = nC = 0
    GCCont = 0
    times = 0
    count = 0
    for line in file:
        count += 1
        if (count > 1): # the first line contains the non-sequence header so we discard it 
            length=len(line)
            total=total+length
            seq=seq+line[0:length-1]

    for k in range(len(seq)):
        if(seq[k]=="G"):
            nG+=1
        elif(seq[k]=="C"):
            nC+=1
    GCContent=(nG+nC)/len(seq)
    return(GCContent)


In [ ]:
EcoliGC = naiveGC('files/ecoli.fa')
StaurGC = naiveGC('files/Staaur.fa')
print(EcoliGC, StaurGC)

#### GCContent: Implementation (using Python's count function)

The above approach is extremely naive as it makes use not so much of an exhaustive search (this needs to be done anyway in order to enumerate all instances), but because of the naiva approach to test each nucleotide against the pattern. 
We ca

In [ ]:
def fastGC(genomefile):

    import regex as re

    file = open(genomefile, 'r')

    seq = ""
    total = 0
    nG = nC = 0
    GCCont = 0
    times = 0
    count = 0
    for line in file:
        count += 1
        if (count > 1):
            length = len(line)
            total = total+length
            seq = seq+line[0:length-1]
    file.close()
    seq = seq.upper()
    
    nC = seq.count("C")
    nG = seq.count("G")
    GCContent = (nG+nC)/len(seq);
    return(GCContent)

In [ ]:
EcoliGC = fastGC('files/ecoli.fa')
StaurGC = fastGC('files/Staaur.fa')
print(EcoliGC, StaurGC)

What we have basically done is use a built-in python function to search and count for all instances of the two nucleotides.
We will come back to this later on. We now have a method to calculate the GC% of a given genome and can move on to some practical analyses. 

#### Using sequence composition to infer phylogeny
In bacteria sequence composition is stable throughout the genome but varies widely between species. One can use simple measures of the nucleotide constitution to infer phylogenetic relationships between species. Below, we will try to do just that, first just by using the GC%.

#### Hands on #1:
* Download a couple of bacterial genome sequences from ENSEMBL Bacteria (http://bacteria.ensembl.org/index.html)
* Implement GC content
* Report the results

* An example would be

Genome | GC |
------------ | -------------
a-Bac1 | 0.334
e-Bac2 | 0.595
e-Bac3 | 0.668
g-Bac4 | 0.409
e-Bac5 | 0.551
a-Bac6 | 0.352
a-Bac7 | 0.354
g-Bac8 | 0.418
g-Bac9 | 0.434
e-Bac8 | 0.627


#### Problem 2: Variability of GC content _between_ Bacterial Genomes
* Given a number of bacterial genomes:  
    - Get their genome sequences  
    - Calculate the GC contents  
    - Calculate differences between the GC contents  
    - Rank genomes based on their differences  
* Pseudocode:
    * Perform GC_content on each of the genomes you downloaded  
    * Calculate D_(i,j)=|GC_i-GC_j| over all i,j  
    * Sort D_(i,j)

#### Problem 2: Approach
* Below we will do this on a set of precalculated GC% values from 10 different bacterial genomes. 


In [ ]:
f=open('files/GCContent.tsv', 'r')

i=0
GCC={}
for line in f:
    i=i+1
    if(i>1):
        species=line.split()[0]
        GC=line.split()[1]
        GCC[species]=float(GC)

gcdistances={}
for genome1 in GCC.keys():
    for genome2 in GCC.keys():
        pair=genome1+":"+genome2
        gcdistances[pair]=abs(float(GCC[genome1])-float(GCC[genome2]))
        gcdistances[pair]=round(gcdistances[pair],2)

sorted(gcdistances.items(), key=lambda x: x[1])


Let's now look how we can use this simple quantity to infer relationships between different genomes. 
In the following, we make use of some python functions to organize the genomes in a tree structure that resembles the way evolutionary biologists try to infer phylogenetic relationships.

In [ ]:
## Clustering of a dataset 

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage, dendrogram

# Load the dataframe and assign values/labels
df = pd.read_csv('files/GCContent_simple.csv')
dvalues = df['GCContent'].values.reshape(-1,1)
dlabels = list(df['Genome'])

# Calculate the distances
distances = pdist(dvalues)

# Convert the pairwise distances into a square distance matrix
distance_matrix = squareform(distances)

# Calculate the linkage matrix using Ward's method
linkage_matrix = linkage(distance_matrix, method='ward')

# Plot the dendrogram
sns.set_style('white')
dendrogram(linkage_matrix, labels=dlabels, color_threshold=0, orientation='left')

# Show the plot
plt.show()

Notice how smaller values are obtained for same bacterial family (a-, g- and e-proteobacteria) which results in them being grouped together in the same clade of the dendrogram.

#### Problem 3: What about different regions of the genome?
* We just saw how genomic GC% values may be used to draw conclusions for bacterial phylogeny
* But: How representative is the GC% value you calculated above?
* And: How efficiently can it be used to describe a genome?

#### Problem 3: Why should we care?
* We mentioned that GC% is stable within bacterial genomes
* **But** Some areas of bacterial genomes are special
* Parts of the bacterial genome have been "horizontally" (as opposed to vertically, i.e. from their "mom") transferred from other species.

![Horizontal gene transfer (HGT)](figures/HGT.png)

#### Problem 3: Stability of GC content _along_ Bacterial Genomes
* Regions of "strange", or "deviating" GC% values in a given genome are red flags of HGT. The problem now is:
    * Given a bacterial genome sequence:  
    * Locate regions of the genome where horizontal gene transfer may have occurred.

#### Problem 3: Approach
* Choose a window to scan your sequence. This will be your resolution.
* Calculate GC per window
* Try to locate GC values that deviate from the genome average

#### Problem 3: The core
* We basically repeat the approach for GC content but now we calculate one value for each window


In [ ]:
def windowGC(genomefile, window):

    import regex as re
    f=open(genomefile, 'r')

    seq = ""
    nG = nC = 0
    total = 0
    window = int(window)

    for line in f:
        x=re.match(">", line)
        if x == None:
            length=len(line)
            total=total+length
            seq=seq+line[0:length-1]
    f.close()
    seq = seq.upper()

    step=int(window/10)-1 # we use a 10% sliding overlap between windows
    times=int(len(seq)/step);

    GCwin = {}
    for i in range(times-10): 
        DNA=seq[i*step:i*step+window]
        nC=DNA.count("C")
        nG=DNA.count("G")
        GCwin[i*step] = (nG+nC)/window
    
    return(GCwin)

In [ ]:
EcoliWGC = windowGC("files/ecoli.fa", 10000)
print(list(EcoliWGC.values())[1:10])

Below we see how we can visualize this in a plot

In [ ]:
import matplotlib.pyplot as plt

EcoliWGC = windowGC("files/ecoli.fa", 10000)
# Create a figure and axis
plt.rcParams['figure.dpi'] = 300
plt.figure(figsize=(8, 6))
# Plot the line
plt.plot(EcoliWGC.keys(), EcoliWGC.values(), label='E.coli', color='blue', linewidth=2)  

# Add labels and title
plt.title('GC content along the genome')
plt.xlabel('genome coordinates')
plt.ylabel('GC%')

# Add a legend
plt.legend()

# Show grid
plt.grid()

# Display the plot
plt.show()

#### Hands-on #2:
* Get the genome sequence of St. aureus
* Implement Sliding GC
* Locate positions in the genome with extreme values of GC
* The problem is: **What do we mean by "extreme values"?** How do we define "extreme"?

#### Problem 3: Statistics Interlude
* Given a set/sample of values, how can we decide on whether a value could be part of that sample or not?
* In our problem: We know that the GC% of bacteria tends to be characteristic of the genome. Can we "spot" regions of the genome that bear GC% values that are _different_ from that characteristic value?
* Q1: How will we define that characteristic value?
* Q2: How will we quantify the _difference_ as big enough or not?

#### Problem 3: Theoretical basis (simplified)
* Central Limit Theorem (simplified):     
    * Regardless of the underlying distribution, the mean of a large number of samples follow the normal distribution.  
    * We can thus model GC values per window based on the normal distribution

#### Problem 3: The statistics
* We will model the "characteristic value" as the mean of GC values for all windows
* We will also calculate the standard deviation of these values to assess the variance
* We will then apply a technique called Z-transformation

#### Z-transformation
* Given a value x, we can compare x to a normal distribution with mean=m and standard deviation=std with the z-score:
    Z(x)= (x - m)/std  
    Z(x) is thus the difference of x from m in units of standard deviation.  
    Knowing that in a normal distribution ~99.5% of the values fall within +/-3*std a value of Z(x)>3 or Z(x)<-3 makes it highly unlikely that x is part of our distribution.

![Normal Distribution](figures/NormalDistribution.jpeg)

#### Problem 3. Predicting HGT locations
* We can now combine sliding GC content calculations with a Z-score transformation and a filtering for |Z|>=3



In [ ]:
def Z_GC(genomefile, window, threshold):
    
    import regex as re
    import numpy as np
    f=open(genomefile, 'r')

    seq = ""
    nG = nC = 0
    total = 0
    window = int(window)

    for line in f:
        x=re.match(">", line)
        if x == None:
            length=len(line)
            total=total+length
            seq=seq+line[0:length-1]
    f.close()

    step=int(window/10) # we use a 10% sliding overlap between windows
    times=int(len(seq)/step);

    GCwin = {}
    for i in range(times): 
        DNA=seq[i*step:i*step+window]
        nC=DNA.count("C")
        nG=DNA.count("G")
        GCwin[i*step] = (nG+nC)/window
    
    GCcont =  list(GCwin.values())
    mGC=np.mean(GCcont)
    sdGC=np.std(GCcont)
    zGC=(GCcont-mGC)/sdGC
    for i in range(len(zGC)):
        if abs(zGC[i]) >= threshold:
            print(i*step, zGC[i])


In [ ]:
Z_GC('files/Staaur.fa', 10000, 5)

In [ ]:
def Z_GC(genomefile, window, threshold):
    import numpy as np
    GCwin = windowGC(genomefile, window)
    GCcont =  list(GCwin.values())
    mGC=np.mean(GCcont)
    sdGC=np.std(GCcont)
    zGC=(GCcont-mGC)/sdGC
    #for i in range(len(zGC)):
    #    if abs(zGC[i]) >= threshold:
    #        print(i*(window/10), zGC[i])
    return(zGC)

Z_GC("files/ecoli.fa", 1000, 3)

In [ ]:
EcoliΖGC = list(Z_GC("files/ecoli.fa", 10000, 3))
print(EcoliΖGC)

In [ ]:
import matplotlib.pyplot as plt

EcoliΖGC = list(Z_GC("files/ecoli.fa", 10000, 3))
# Create a figure and axis
plt.rcParams['figure.dpi'] = 300
plt.figure(figsize=(8, 6))
# Plot the line
plt.plot(EcoliWGC.keys(), EcoliΖGC, label='E.coli', color='blue', linewidth=2)  

plt.axhline(y=-3, color='red', linestyle='--', label='Lower Boundary')
plt.axhline(y=3, color='red', linestyle='--', label='Upper Boundary')

# Add labels and title
plt.title('Z-GC content along the genome')
plt.xlabel('genome coordinates')
plt.ylabel('z-GC%')

# Add a legend
plt.legend()

# Show grid
plt.grid()

# Display the plot
plt.show()

#### Bonus Question

In the above output we see a lot of reported values coming from consecutive indices (i.e. coordinates that are adjacent to each other). 
Complete the function Z_GC() so that it reports ranges, that is genomic intervals that are supported by the **mean Z-score** of the range. 

#### Problem 2: Revisited

* Background DNA composition has some **functional** role besides simply reflecting mutational pressures
* This means that in some cases we need to know why the local composition is guided by _other_ aspects of molecular evolution. e.g. why would rRNA genes be G+C-rich even in AT-rich genomes?
* We need to find a way to control for _background nucleotide composition_

#### Problem 2 Revisited: Distinguishing between genomes through their sequence composition
1. Going beyond the GC content
2. Going beyond simple bases (mononucleotides, k=1)
3. Analyzing all dinucleotide frequencies of k=2

* Pseudocode:
    * For each kmer in _4^k_ k-mers
    * Calculate N(kmer)
    * Create a table



#### Problem: How to count k-mer frequencies
* For mononucleotides we did it with a Brute Force approach. However the mononucleotides are 4. The k-mers are 4^k.
* How can we count the frequencies of k-mers?



In [ ]:
def allkmers(genomefile, k):
    # genomefile is a fasta file with one sequence
    # k is the length of kmers you want to calculate
    # example call: kmers("myfasta.fa", 5) for all existing 5-mers in myfasta.fa
    import regex as re
    import itertools

    # Importing sequence
    file = open(genomefile, 'r')
    seq = ""
    count = 0
    for line in file:
        count +=1
        if (count > 1) :
            length=len(line)
            seq=seq+line[0:length-1]
            
    file.close()

    seq = re.sub("[^AGCT]", "", seq)

    # Creating the kmer combination table
    nucleotides = ['A', 'C', 'G', 'T']
    mykmers = [''.join(p) for p in itertools.product(nucleotides, repeat=k)]

    # Counting kmer occurrence in the sequence
    kmertable = {} 
    for i in mykmers:
        kmertable[i]=len([m.start() for m in re.finditer(rf'(?={str(i)})', seq)])

    kmertable = {k: float(v) / len(seq) for k, v in kmertable.items()}
    return(kmertable)
    


In [ ]:
    
allkmers('files/ecoli.fa', 2)

Now let's try to do this for k=5

In [ ]:
import time
start_time = time.time()

allkmers('files/ecoli.fa', 5)

print("--- %s seconds ---" % (time.time() - start_time))



### Stop and think:
  1. Do we need **all** k-mers?
  2. Do we need to check each k-mer at every step?
* How many calculations do we need if we answer "yes" to 1,2 above.


#### Solution: Hashing Strategy instead of Brute Force pattern comparisons
* Read the sequence in chunks of _k_ nucleotides
* For each subsequence increment a dictionary value with the subsequence as key

#### Problem 2 Revisited: K-mer frequencies


In [ ]:
def hashkmers(genomefile, k):
    # genomefile is a fasta file with one sequence
    # k is the length of kmers you want to calculate
    # example call: kmers("myfasta.fa", 5) for all existing 5-mers in myfasta.fa
    import regex as re

    file = open(genomefile, 'r')

    seq = ""
    kmertable = {} 

    count = 0
    for line in file:
        count +=1
        if (count > 1) :
            length=len(line)
            seq=seq+line[0:length-1]
            
    file.close()

    seq = re.sub("[^AGCT]", "", seq)

    for i in range(len(seq)-k):
        DNA=seq[i:i+k]
        if DNA not in kmertable.keys():
            kmertable[DNA]=1
        else:
            kmertable[DNA]+=1

    kmertable = {k: float(v) / len(seq) for k, v in kmertable.items()}
    kmertable = sorted(kmertable.items()) #sorting by key (alphabetically)
    return(kmertable)


In [ ]:
hashkmers('files/ecoli.fa', 2)

Compare the hashing strategy with the allkmers strategy in the case of 5-mers (k=5)

In [ ]:
import time
start_time = time.time()

hashkmers('files/ecoli.fa', 5)

print("--- %s seconds ---" % (time.time() - start_time))

**Conclusion**

Taking the complexity into account is imperative in the case of combinations. 

#### Problem 2 Revisited: A table of _4^k_ frequencies of occurrence  
Base | A | T | G | C
------------- | ------------- | ------------- | ------------- | -------------
A | 0.090 | 0.112 | 0.048 | 0.053
T | 0.095 | 0.090 | 0.064 | 0.053
G | 0.052 | 0.052 | 0.023 | 0.034
C | 0.066 | 0.048 | 0.026 | 0.023

* Values may be seen as "probabilities" of finding each k-mer in the sequence
* Can we use the notion of the probability to modify the table so that we get rid of the background nucleotide composition?

#### Problem 2 Revisited: Removing Background Composition
* The problem stated above persists at the level of k-mers: The background DNA composition may affect our results
* At the k-mer level we can remove the background using ratios of observed/expected frequencies
* Which is the expected frequency of a given k-mer?

#### Problem 2 Revisited: Observed/Expected(o/e) k-mer frequencies
* Mathematics Interlude:
    * Assume two events A, B that are linked with each other
    * We then say that A and B are dependent (or conditioned) and we have a "conditional probability" of A happening given B is also happening
    * We can think of k-mers the same way: a k-mer is more probable to occur if its constituent mono-mers are occurring
    * Bottomline: Any given k-mer's frequency of occurrence is dependent on the frequencies of occurrence of its mononucleotides. Thus:

Given a k-mer of length k the o/e-ratio frequency is defined as:  
  $$R[N_1N_2..N_k]=F[N_1N_2..N_k]/(F[N_1]F[N_2]..F[N_k])$$

In this way we can define a new table of modified frequencies that is independent of mono-nucleotide composition

#### Problem 2 Revisited: Observed/Expected K-mer frequencies


In [ ]:
def rkmers(genomefile, k):
    import regex as re

    file = open(genomefile, 'r')

    seq = ""
    kmertable = {} 

    count = 0
    for line in file:
        count +=1
        if (count > 1) :
            length=len(line)
            seq=seq+line[0:length-1]
            
    file.close()

    seq = re.sub("[^AGCT]", "", seq)
    
    # calculation of background nucleotide probability
    pnuc = {}
    pnuc['A'] = float(seq.count('A')/len(seq))
    pnuc['C'] = float(seq.count('C')/len(seq))
    pnuc['G'] = float(seq.count('G')/len(seq))
    pnuc['T'] = float(seq.count('T')/len(seq))
    GCcont=pnuc['G']+pnuc['C']
    
    for i in range(len(seq)-k):
        DNA=seq[i:i+k]
        if DNA not in kmertable.keys():
            kmertable[DNA]=1
        else:
            kmertable[DNA]+=1

    kmertable = {k: float(v) / len(seq) for k, v in kmertable.items()}
    
    rkmertable = kmertable
    for kmer in kmertable.keys():
        pkmer=1
        for j in range(len(kmer)):
            nuc = list(kmer)[j]
            pkmer = pkmer * pnuc[nuc]
        rkmertable[kmer]=round(kmertable[kmer]/(pkmer),3)
    
    return(rkmertable)


In [ ]:
rkmers('files/ecoli.fa', 2)

#### Problem 2 Revisited: A table of o/e 4^k frequencies of occurrence  
Base | A | G | C | T
------------- | ------------- | ------------- | ------------- | -------------
A | 0.800 | 0.997 | 0.878 | 0.949
G | 0.848 | 0.799 | 1.174 | 0.957
C | 0.946 | 0.955 | 0.848 | 1.252
T | 1.183 | 0.872 | 0.946 | 0.841

* Notice how values now go >1. What does this mean?
* How is this table better (or not) than the previous one?

#### Genomic Signatures: Comparing o/e k-mer composition
* Genomic Signatures are defined as the table of o/e k-mers for a given genome
* We can use these tables to analyze distances between genomes. (Hint: even eukaryote genomes!)

#### Hands-on #3:
* Get chromosome 1 from (human, mouse, fly, worm, yeast)
* Use a genomic signature approach to cluster genomic signatures from different genomes
* Calculate the distance between rho_xy(p) and rho_xy(s) to create a table of distances.

#### Problem 4: Finding the DNA Replication in a bacterial genome
* Bacterial Genomes replicated their genome starting at one point and proceeding towards the opposite point in  the circular genome from both directions.
* Bacterial genomes also have a particular distribution of nucleotides along their genome
* The difference of A-T (and G-C) complementary nucleotides goes through a sort of "phase transition" that splits the genome approximately in half.
* Do you know what this split is?
* Do you know why it is so?

#### How is this related to Sequence Analysis?
* Due to the pioneering work of E. Chargaff we know that A~T and G~C in **single-stranded DNA**
* We know that this holds for all complete genomes except very few exceptions
* The exceptions are the few genomes that **do not** replicate symmetrically
* DNA-strand parity:
    * Strand X is replicated in-continuously
    * Accumulates more substitutions
    * If substitutions are biased the strand will guide the change in both strands through base-pairing

![Genome of Staphylococcus aureus](figures/Figure00_02.jpg)

#### Approaching the problem
* We thus expect (and observe) the parity to be violated and that this violation occurs symmetrically on either side of the OriC
* We are looking for a way to locate this _transition_ in the parity violation
* We thus need:
    * A measure of the parity
    * A way to monitor this measure along the genome
    * A way to locate abrupt changes in its values

#### Breaking the problem into pieces
1. Analyze the DNA composition _along_ the genome
2. Calculate a quantity that will be informative
3. Create a condition that will test the location of the Ori

* Pseudocode: Given a bacterial genome:
    * Count nucleotides in windows of N base pairs
    * Calculate the scaled AT-skew as (A-T)/(A+T)
    * Create an array of the skew values along the genome
    * Locate the transition point

#### Problem 4: Parity Measure Implementation


In [ ]:
f=open('files/Staaur.fa', 'r')

import regex as re

seq = ""

for line in f:
    x=re.match(">", line)
    if x == None:
        length=len(line)
        seq=seq+line[0:length-1]
f.close()

window=1000
step=100
times=int(len(seq)/step)

ATparity = []
for i in range(times):
    DNA=seq[i*step:i*step+window]
    A=DNA.count("A")
    T=DNA.count("T")
    C=DNA.count("C")
    G=DNA.count("G")
    ATparity.append(float(A-T)/float(A+T))

for k in range(10):
    print(k,":",ATparity[k])

an alternative of the above for reading the fasta file with Pythons SeqIO function from the BIO library

In [ ]:
from Bio import SeqIO

fasta_sequences = SeqIO.parse(open(input_file),'fasta')
with open(output_file) as out_file:
    for fasta in fasta_sequences:
        name, sequence = fasta.id, fasta.seq.tostring()
        new_sequence = some_function(sequence)
        write_fasta(out_file)

#### Problem 4: Plotting the Values

In [ ]:
import matplotlib.pyplot as plt
import regex as re  

f = open('files/Staaur.fa', 'r')
seq = ""
total = 0
A=T=G=C=[]
times=0;
for line in f:
	x=re.match(">", line)
	if x == None:
		length=len(line)
		total=total+length
		seq=seq+line[0:length-1]
f.close()

x=[]
ATparity=[]
window=100000
step=10000
times=int(len(seq)/step);
for i in range(times):
    x.append(i*step)
    DNA=seq[i*step:i*step+window]
    A=DNA.count("A")
    T=DNA.count("T")
    C=DNA.count("C")
    G=DNA.count("G")
    ATparity.append(float(A-T)/float(A+T))

# plotting points as a scatter plot
plt.plot(x, ATparity, color= "green", linewidth = 3.0)
plt.axhline(y = 0, color = 'grey', linewidth = 2.0)

# plotting vertical lines at Ori and Ter
plt.axvline(x = 1080000, color = 'red')
plt.axvline(x = 2450000, color = 'blue')

#plt.scatter(x, ATparity, color= "green")

# x-axis label
plt.xlabel('Genome Coordinates')
# frequency label
plt.ylabel('(A-T)/(A+T)')
# plot title
plt.title('S. aureus AT parity')
# showing legend
#plt.legend()

# function to show the plot
plt.show()


#### Problem 4: Locating the breakpoint(s)
* Not a simple problem. In fact one (breakpoint detection) for which research is ongoing in many fields
* Things you could try:
    * Using derivation (checking the difference between each value and the previous one)
    * Density-based approaches: Trying to locate the region around which changes in the sign occur more robustly (i.e. given many different points around it)

#### Concept. Binary Searches   (revisit this!)
* Let's think of a simpler problem first:
Suppose you are given a quadratic equation: f(x)=ax**2+bx+c and you are asked to locate a root of the equation in an interval [k,m].
* How would you proceed?
* A fast and efficient way is to start by checking the values f(k) and f(m). If their product in f(k)f(m)<0 this means that the function "crosses" the x-axis at some point between k and m. (This stems from Lagrange's mean value theorem). The question is how then can we locate that point?
* What we have in our hands is quite similar. We know that AT (or GC) parity values are expected to change their sign at two points in the genome (coinciding with the origin and terminus of replication). We need a way to locate this sign-transitions in a fast and accurate manner.

#### Take a pause and think. 
1. How would you do it with an exhaustive (brute-force) approach?
2. How would you try to do it with a divide-and-conquer approach

Following, we show a function that takes as input the genomefile, a window of parity calculation, the type of parity (AT or GC) and one additional parameter, which we call "threshold" and which we will use to assess the proximity to a transition point.

In [ ]:
### A brute-force with a threshold

def bruteParity(genomefile, window, type, threshold):
    
    import matplotlib.pyplot as plt
    import regex as re  

    f = open(genomefile, 'r')
    seq = ""
    total = 0
    A=T=G=C=[]
    times=0;
    for line in f:
        x=re.match(">", line)
        if x == None:
            length=len(line)
            total=total+length
            seq=seq+line[0:length-1]
    f.close()

    x = []
    parity = []
    transitions = []
    step = int(window/10)
    times = int(len(seq)/step);
    for i in range(times):
        x.append(i*step)
        DNA=seq[i*step:i*step+window]
        A=DNA.count("A")
        T=DNA.count("T")
        C=DNA.count("C")
        G=DNA.count("G")
        if type == "AT":
            parity.append(float(A-T)/float(A+T))
            if (abs(float(A-T)/float(A+T)) < threshold):
                transitions.append(i*step)
        if type == "GC":
            parity.append(float(G-C)/float(G+C))
            if (abs(float(G-C)/float(G+C)) < threshold):
                transitions.append(i*step)
    
    # Plot Parity and Predicted Transitions
    plt.plot(x, parity, color= "green", linewidth = 3.0)
    plt.axhline(y = 0, color = 'grey', linewidth = 2.0)

    # plotting vertical lines at Ori and Ter
    for site in transitions:
        plt.axvline(x = site, color = 'black')

    #plt.scatter(x, parity, color= "green")

    # x-axis label
    plt.xlabel('Genome Coordinates')
    # frequency label
    plt.ylabel('Relative Difference')
    # plot title
    plt.title('Parity')
    # showing legend
    #plt.legend()

    # function to show the plot
    plt.show()

    return(transitions)


What we do in this case is very simple:
1. We go on by calculating the parity in a given window
2. We set an absolute value threshold below which we consider the parity to be zero
3. We ask the script to report the positions were this is true. 

In [ ]:
bruteParity('files/Staaur.fa', 100000, 'AT', 0.01)

We see that we locate the two main transition points plus an extra one. We actually have to tune the resolution parameter a bit to get the correct positions.

In [ ]:
bruteParity('files/Staaur.fa', 100000, 'AT', 0.005)

Depending on the genome and its shape of the parity profile we need to tune the parameters accordingly and, obviously, this is not a good strategy for unknown genomes. Let's see for instance how the same parameteres behave in the case of E. coli.

In [ ]:
bruteParity('files/ecoli.fa', 100000, 'AT', 0.005)

AT is not a good choice for E. coli, so we switch to GC parity, for which, relaxing the threshold a bit (at 0.01) allows us to define the major transition point. In this case, the other point coincides with the beginning and end of the sequence.

In [ ]:
bruteParity('files/ecoli.fa', 100000, 'GC', 0.01)

The question is, now, if this could be improved in a way that would not require tuning of parameters except from the choice of parity profile (AT or GC). 

#### A Divide and Conquer approach (think about it)
We can think of a way that proceeds iteratively by narrowing down the space where the transition is being searched for. One such approach would be to roughly estimate the location of the transition in a range and then proceed by splitting that range in smaller and smaller pieces. This could work in the case of a single transition as is the one of E. coli.


#### Exercises
1. Use a genomic signature approach to locate possible HGT genes in the genome of _St. aureus_. Do your results of "outliers" differ from those obtained with the GC content approach?
2. Employ a similar approach for HGT detection but using Genomic Signatures for k=2 or k=3.
3. Write a faster program to locate the origin of replication for a given bacterial genome using the parity rules described in the lecture.


### Extras (KEDIVIM)

In [ ]:
import matplotlib.pyplot as plt

hgWGC = windowGC("files/hg19_chr1_part.fa", 10000)
# Create a figure and axis
plt.rcParams['figure.dpi'] = 300
plt.figure(figsize=(8, 6))
# Plot the line
plt.plot(hgWGC.keys(), hgWGC.values(), label='H. sapiens, chr1', color='blue', linewidth=2)  

# Add labels and title
plt.title('GC content along the genome')
plt.xlabel('genome coordinates')
plt.ylabel('GC%')

# Add a legend
plt.legend()

# Show grid
plt.grid()

# Display the plot
plt.show()

In [ ]:
import time
start_time = time.time()

allkmers('files/hg19_chr1_part.fa', 2)

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
hg2mers = rkmers('files/hg19_chr1_part.fa', 2)
type(hg2mers)
print(hg2mers)

In [ ]:
import matplotlib.pyplot as plt

hgk = rkmers('files/hg19_chr1_part.fa', 2)
hg2mers = dict(sorted(hg2mers.items(), key=lambda item: item[1]))

# Extract keys and values from the dictionary
items = list(hg2mers.keys())
values = list(hg2mers.values())

# Create a bar plot
plt.figure(figsize=(10, 6))
plt.bar(items, values, color='skyblue')

# Add labels and title
plt.xlabel('k-mers')
plt.ylabel('Frequency')
plt.title('Frequencies of Occurrence (k=2, Homo sapiens)')

# Show grid for better readability
plt.grid(axis='y')

# Display the plot
plt.show()

In [ ]:
def readfasta(fastafile):
    import regex as re
    f=open(fastafile, 'r')
    seq = ""
    total = 0
    for line in f:
        x=re.match(">", line)
        if x == None:
            length=len(line)
            total=total+length
            seq=seq+line[0:length-1]
    seq=seq.replace('N','')
    f.close()
    return(seq)

def CpGfinder(seq):
    C=seq.count("C")/len(seq)
    G=seq.count("G")/len(seq)
    CpG=seq.count("CG")

    GCcont=float(G+C)
    CpGobs=float(CpG/(len(seq)-1))
    rCpG = float(CpGobs/(C*G))

    return(GCcont, rCpG)

def scanSeq(seq, window, step):
    times=int(len(seq)/step)
    x = []
    for i in range(times):
        x.append(i*step)
        DNA=seq[i*step:i*step+window]
        results=CpGfinder(DNA)
        if((results[0]>=0.5) & (results[1]>=0.6)):
            print(i*step,"-",i*step+200, results)

scanSeq(readfasta("files/hg19_chr1_part.fa"), 10000, 1000)

In [ ]:
def needleman_wunsch(seq1, seq2, match_score=1, mismatch_penalty=0, gap_penalty=-2):
    # Initialize the scoring matrix
    n = len(seq1)
    m = len(seq2)
    
    # Create a (n+1) x (m+1) matrix
    score_matrix = [[0] * (m + 1) for _ in range(n + 1)]
    
    # Initialize the first row and column of the matrix
    for i in range(n + 1):
        score_matrix[i][0] = i * gap_penalty
    for j in range(m + 1):
        score_matrix[0][j] = j * gap_penalty

    # Fill the scoring matrix
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            match = score_matrix[i - 1][j - 1] + (match_score if seq1[i - 1] == seq2[j - 1] else mismatch_penalty)
            delete = score_matrix[i - 1][j] + gap_penalty
            insert = score_matrix[i][j - 1] + gap_penalty
            score_matrix[i][j] = max(match, delete, insert)

    # Trace back to find the optimal alignment
    aligned_seq1 = []
    aligned_seq2 = []
    
    i, j = n, m
    while i > 0 or j > 0:
        current_score = score_matrix[i][j]
        
        if i > 0 and j > 0:
            if current_score == score_matrix[i - 1][j - 1] + (match_score if seq1[i - 1] == seq2[j - 1] else mismatch_penalty):
                aligned_seq1.append(seq1[i - 1])
                aligned_seq2.append(seq2[j - 1])
                i -= 1
                j -= 1
                continue
        
        if i > 0 and current_score == score_matrix[i - 1][j] + gap_penalty:
            aligned_seq1.append(seq1[i - 1])
            aligned_seq2.append('-')
            i -= 1
        else:
            aligned_seq1.append('-')
            aligned_seq2.append(seq2[j - 1])
            j -= 1

    # Reverse the aligned sequences since we traced back
    aligned_seq1.reverse()
    aligned_seq2.reverse()

    return ''.join(aligned_seq1), ''.join(aligned_seq2), score_matrix[n][m]

# Example usage
if __name__ == "__main__":
    sequence_1 = "ATTCCGCCCAATTACGGA"
    sequence_2 = "ATCGCCCAAACGGA"

    alignment_1, alignment_2, final_score = needleman_wunsch(sequence_1, sequence_2)
    
    print("Alignment:")
    print(alignment_1)
    print(alignment_2)
    print("Final Score:", final_score)

In [ ]:
def burrows_wheeler_transform(input_string):
    # Append a unique end character to the input string
    input_string += '$'
    
    # Generate all cyclic rotations of the input string
    rotations = [input_string[i:] + input_string[:i] for i in range(len(input_string))]
    
    # Sort the rotations lexicographically
    sorted_rotations = sorted(rotations)
    
    # Extract the last column from the sorted rotations
    last_column = ''.join(rotation[-1] for rotation in sorted_rotations)
    
    # Find the index of the original string in the sorted rotations
    original_index = sorted_rotations.index(input_string)
    
    return last_column, original_index

# Example usage
if __name__ == "__main__":
    input_text = "ATGCACACTTTACACATGGG"
    bwt_result, index = burrows_wheeler_transform(input_text)
    
    print("Input text: ", input_text)
    print("Burrows-Wheeler Transform: ", bwt_result)
    print("Index of original string in sorted rotations: ", index)

In [ ]:
def burrows_wheeler_decode(last_column, original_index):
    # Step 1: Create a table to hold the sorted characters
    n = len(last_column)
    
    # Create a list of tuples (character, original index)
    table = [(last_column[i], i) for i in range(n)]
    
    # Step 2: Sort the table by the first element (the character)
    table.sort()

    # Step 3: Create an array to hold the first column
    first_column = [table[i][0] for i in range(n)]

    # Step 4: Create an array to track the next indices
    next_indices = [0] * n
    for i in range(n):
        next_indices[i] = table.index((last_column[i], i))

    # Step 5: Reconstruct the original string
    decoded_string = []
    index = original_index
    
    for _ in range(n):
        decoded_string.append(last_column[index])
        index = next_indices[index]

    return ''.join(decoded_string[::-1])  # Reverse to get the original string

# Example usage
if __name__ == "__main__":
    last_column = "GTCCC$CGAAAAGTGTTAAT"  # Example last column from BWT
    original_index = 3        # Example index of original string in sorted rotations

    decoded_text = burrows_wheeler_decode(last_column, original_index)
    
    print("Decoded Text:", decoded_text)